In [1]:
import numpy as np
import pandas as pd
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials 
# import time
import joblib
# import math
%matplotlib inline

# load the album data 

In [2]:
album_dir = 'D:\\Data\\spotify_albums\\'
master_albums = joblib.load( album_dir+ 'master_albums_spotify.pkl')  
master_albums.shape

(485, 9)

# load the album meta-data

In [29]:
album_meta_data_dir = 'D:\\Data\\spotify_album_meta_data\\'
master_album_meta_data = joblib.load(album_meta_data_dir + 'album_meta_data.pkl')
# all genre info is missing
master_album_meta_data.drop('genres_sp', axis=1, inplace=True)
master_album_meta_data.shape

(485, 4)

# load the track data

In [3]:
track_dir = 'D:\\Data\\spotify_album_tracks\\'
master_tracks = joblib.load(track_dir + 'omnibus_album_tracks.pkl')  
master_tracks.shape

(6901, 9)

# load the musical features per track

In [7]:
# # save it out
musical_features_dir = 'D:\\Data\\spotify_album_track_features\\'
master_track_features = joblib.load(musical_features_dir + 'track_features.pkl')  
master_track_features.rename(columns={'uri': 'track_uri'}, inplace=True)
master_track_features.shape

(6901, 18)

# Part 1: Merge album and album meta-data

In [23]:
master_albums.head()

,album_to_search,artist_to_search,artist_name_sp,artist_uri_sp,album_name_sp,uri_sp,total_tracks_sp,release_date_sp,release_type_sp
0,÷,ed sheeran,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2017-03-03,album
1,remixed,nina simone,Nina Simone,spotify:artist:7G1GBhoKtEPnP86X2PvEYO,Remixed & Reimagined,spotify:album:0PJ70lJhWjflyKkrQabNso,16,2006-10-31,album
2,routine maintenance,aaron west and the roaring twenties,Aaron West and The Roaring Twenties,spotify:artist:59cc2f0IvGu6YVEtY4cS0p,Routine Maintenance,spotify:album:5xAEfL4n6fBI5WZ9ugrmIw,10,2019-05-10,album
3,25,adele,Adele,spotify:artist:4dpARuHxo51G3z768sgnrY,25,spotify:album:0K4pIOOsfJ9lK8OjrZfXzd,11,2016-06-24,album
4,the impossible kid,aesop rock,Aesop Rock,spotify:artist:2fSaE6BXtQy0x7R7v9IOmZ,The Impossible Kid,spotify:album:1An1m0S3ZdQy9Uuo476D12,15,2016-04-29,album


In [35]:
master_album_meta_data.head()

,uri_sp,album_name,popularity_sp,label_sp
0,spotify:album:3T4tUhGYeRNVUGevb0wThu,÷ (Deluxe),89,Atlantic Records UK
1,spotify:album:0PJ70lJhWjflyKkrQabNso,Remixed & Reimagined,41,RCA/Legacy
2,spotify:album:5xAEfL4n6fBI5WZ9ugrmIw,Routine Maintenance,41,Hopeless Records
3,spotify:album:0K4pIOOsfJ9lK8OjrZfXzd,25,77,XL Recordings/Columbia
4,spotify:album:1An1m0S3ZdQy9Uuo476D12,The Impossible Kid,55,Rhymesayers


In [34]:
# [x for x in master_album_meta_data.genres_sp]

In [25]:
print(master_albums.uri_sp.isin(master_album_meta_data.uri_sp).sum())
print(master_album_meta_data.uri_sp.isin(master_albums.uri_sp).sum())

485
485


In [36]:
len(np.unique(master_album_meta_data.uri_sp))

479

In [37]:
len(np.unique(master_albums.uri_sp))

479

In [30]:
master_album_meta_data.duplicated().sum()

6

In [38]:
master_albums[master_albums.duplicated()]

,album_to_search,artist_to_search,artist_name_sp,artist_uri_sp,album_name_sp,uri_sp,total_tracks_sp,release_date_sp,release_type_sp
451,american football,american football,American Football,spotify:artist:5FwydyGVcsQllnM4xM6jw4,American Football,spotify:album:3wRBlpk5PRoixwOnLujTal,9,1999,album
457,crystal silence,chick corea,Chick Corea,spotify:artist:5olDKSsFhhmwh8UCWwKtpq,The New Crystal Silence,spotify:album:4w7zx8OyGJgp7WWMACQJvN,13,2008-01-01,album
459,best of,creedence clearwater revival,Creedence Clearwater Revival Experience,spotify:artist:6ziJTSPiXQmTs8AdEFLsPC,The Best of Clearwater Revival Experience,spotify:album:4lFljj38Jam7LyxgAruE0O,12,2019-10-04,album


In [32]:
master_album_meta_data[master_album_meta_data.uri_sp == 'spotify:album:2fSni1iwcCiNlgiZCdUO65']

,uri_sp,album_name,popularity_sp,label_sp
401,spotify:album:2fSni1iwcCiNlgiZCdUO65,The Slackers Live at Sugarshack Sessions,23,Sugarshack
402,spotify:album:2fSni1iwcCiNlgiZCdUO65,The Slackers Live at Sugarshack Sessions,23,Sugarshack


In [39]:
master_albums_no_dupes = master_albums.drop_duplicates(subset = 'uri_sp').reset_index(drop = True)
master_album_meta_data_no_dupes = master_album_meta_data.drop_duplicates().reset_index(drop = True)
print(master_albums_no_dupes.shape)
print(master_album_meta_data_no_dupes.shape)

(479, 9)
(479, 4)


In [41]:
album_level_data = pd.merge(master_albums_no_dupes, master_album_meta_data_no_dupes, how = 'inner', on = 'uri_sp')
print(master_albums_no_dupes.shape)
print(master_album_meta_data_no_dupes.shape)
print(album_level_data.shape)

(479, 9)
(479, 4)
(479, 12)


In [42]:
album_level_data.head()

,album_to_search,artist_to_search,artist_name_sp,artist_uri_sp,album_name_sp,uri_sp,total_tracks_sp,release_date_sp,release_type_sp,album_name,popularity_sp,label_sp
0,÷,ed sheeran,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK
1,remixed,nina simone,Nina Simone,spotify:artist:7G1GBhoKtEPnP86X2PvEYO,Remixed & Reimagined,spotify:album:0PJ70lJhWjflyKkrQabNso,16,2006-10-31,album,Remixed & Reimagined,41,RCA/Legacy
2,routine maintenance,aaron west and the roaring twenties,Aaron West and The Roaring Twenties,spotify:artist:59cc2f0IvGu6YVEtY4cS0p,Routine Maintenance,spotify:album:5xAEfL4n6fBI5WZ9ugrmIw,10,2019-05-10,album,Routine Maintenance,41,Hopeless Records
3,25,adele,Adele,spotify:artist:4dpARuHxo51G3z768sgnrY,25,spotify:album:0K4pIOOsfJ9lK8OjrZfXzd,11,2016-06-24,album,25,77,XL Recordings/Columbia
4,the impossible kid,aesop rock,Aesop Rock,spotify:artist:2fSaE6BXtQy0x7R7v9IOmZ,The Impossible Kid,spotify:album:1An1m0S3ZdQy9Uuo476D12,15,2016-04-29,album,The Impossible Kid,55,Rhymesayers


## Part 1A: Merge in genres from mp3 id3 tags

In [43]:
in_dir_id3 = 'D:\\Data\\Music_Library\\'

In [45]:
tag_data = pd.read_csv(in_dir_id3 + 'tag_data_clean.csv')
# recode stupid ed sheeran album
tag_data.album_clean[tag_data.album_clean.isnull() ] = '÷'

C:\Users\Mike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
tag_data_unique = tag_data.drop_duplicates(subset = ['artist_clean', 'album_clean']).reset_index(drop = True)
tag_data_unique.shape

(792, 12)

In [47]:
tag_data_no_dupes = tag_data_unique.drop_duplicates(subset = 'album_clean', keep = 'first').reset_index(drop = True)
tag_data_no_dupes = tag_data_no_dupes.replace(np.nan, '', regex=True)
tag_data_no_dupes.shape

(615, 12)

In [48]:
tag_data_no_dupes.head()

,album,title,artist,albumartist,tracknumber,genre,date,album_path,length,master_artist,artist_clean,album_clean
0,÷,Eraser,Ed Sheeran,Ed Sheeran,1/12,Pop,2017,D:\Music\- Ed Sheeran\Ed Sheeran\÷\01 - Eraser...,,Ed Sheeran,ed sheeran,÷
1,Remixed & Reimagined,I Can't See Nobody (Daniel Y. Remix),Nina Simone,,1,,,D:\Music\10386_Nina_Simone_Remixed_MP3\01. Nin...,,Nina Simone,nina simone,remixed
2,Routine Maintenance,Lead Paint & Salt Air,Aaron West and The Roaring Twenties,Aaron West and The Roaring Twenties,1,,2019,D:\Music\Aaron West and The Roaring Twenties -...,,Aaron West and The Roaring Twenties,aaron west and the roaring twenties,routine maintenance
3,ABBA Gold: Greatest Hits,Chiquitita,ABBA,ABBA,11,Pop,1992,D:\Music\ABBA\ABBA GOLD - Greatest Hits HQ 320...,326000,ABBA,abba,abba gold greatest hits
4,More ABBA Gold: More ABBA Hits,Summer Night City,ABBA,ABBA,1,Rock,1993,D:\Music\ABBA\More ABBA Gold- More ABBA Hits\0...,210666,ABBA,abba,more abba gold more abba hits


In [55]:
tag_data_no_dupes = tag_data_no_dupes[['artist_clean', 'album_clean', 'genre']]
tag_data_no_dupes.rename(columns={'artist_clean': 'artist_to_search', 'album_clean': 'album_to_search'}, inplace=True)

In [56]:
tag_data_no_dupes.head()

,artist_to_search,album_to_search,genre
0,ed sheeran,÷,Pop
1,nina simone,remixed,
2,aaron west and the roaring twenties,routine maintenance,
3,abba,abba gold greatest hits,Pop
4,abba,more abba gold more abba hits,Rock


In [67]:
album_level_data_final = pd.merge(album_level_data, tag_data_no_dupes, 
                                  how = 'left', on = ['artist_to_search', 'album_to_search'] )
album_level_data_final.rename(columns={'uri_sp': 'album_uri'}, inplace=True)
print(album_level_data.shape)
print(tag_data_no_dupes.shape)
print(album_level_data_final.shape)

(479, 12)
(615, 3)
(479, 13)


In [68]:
album_level_data_final.head()

,album_to_search,artist_to_search,artist_name_sp,artist_uri_sp,album_name_sp,album_uri,total_tracks_sp,release_date_sp,release_type_sp,album_name,popularity_sp,label_sp,genre
0,÷,ed sheeran,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop
1,remixed,nina simone,Nina Simone,spotify:artist:7G1GBhoKtEPnP86X2PvEYO,Remixed & Reimagined,spotify:album:0PJ70lJhWjflyKkrQabNso,16,2006-10-31,album,Remixed & Reimagined,41,RCA/Legacy,
2,routine maintenance,aaron west and the roaring twenties,Aaron West and The Roaring Twenties,spotify:artist:59cc2f0IvGu6YVEtY4cS0p,Routine Maintenance,spotify:album:5xAEfL4n6fBI5WZ9ugrmIw,10,2019-05-10,album,Routine Maintenance,41,Hopeless Records,
3,25,adele,Adele,spotify:artist:4dpARuHxo51G3z768sgnrY,25,spotify:album:0K4pIOOsfJ9lK8OjrZfXzd,11,2016-06-24,album,25,77,XL Recordings/Columbia,Pop
4,the impossible kid,aesop rock,Aesop Rock,spotify:artist:2fSaE6BXtQy0x7R7v9IOmZ,The Impossible Kid,spotify:album:1An1m0S3ZdQy9Uuo476D12,15,2016-04-29,album,The Impossible Kid,55,Rhymesayers,Rap & Hip-Hop


In [69]:
# only 32 albums missing genre!
album_level_data_final.isnull().sum()

album_to_search      0
artist_to_search     0
artist_name_sp       0
artist_uri_sp        0
album_name_sp        0
album_uri            0
total_tracks_sp      0
release_date_sp      0
release_type_sp      0
album_name           0
popularity_sp        0
label_sp             0
genre               32
dtype: int64

In [70]:
album_level_data_final.genre.value_counts()

Rock                            77
                                43
Alternative                     42
Jazz                            38
Rap & Hip-Hop                   20
                                ..
Country | Blues | Folk           1
???                              1
Sub-Saharan Africa               1
Latin Jazz                       1
Flemish Melancholy Indie Pop     1
Name: genre, Length: 81, dtype: int64

In [71]:
[x for x in set(album_level_data_final.genre)]

['',
 'Contemporary Hip Hop',
 'R&B/Soul',
 nan,
 'Country, Singer/Songwriter',
 'Latin Jazz',
 'Rap & Hip-Hop',
 'Hip-Hop/Rap',
 'CHILD',
 'Rap/Hip Hop',
 'Smooth Jazz',
 'Fusion',
 'Flemish Melancholy Indie Pop',
 'Alternatif',
 'Classical Period',
 'Americana',
 'Classical',
 'Classic Rock',
 'Emo',
 'Alternative',
 'Dixieland / Traditional',
 'R&B / Soul / Hip-Hop',
 'Miscellaneous',
 'Pop',
 'Indie',
 'Hip Hop/Rap',
 'Flemish Rock',
 'Soul and R&B',
 'Children',
 'Traditional Country',
 'Sub-Saharan Africa',
 'Downtempo',
 'R&B',
 'Traditional American',
 'Folk',
 'Electronic',
 'Western Europe',
 'Contemporary Folk',
 'Soul',
 'Contemporary Jazz',
 'Contemporary Blues',
 'Hip-Hop',
 'Soundtrack',
 '???',
 'Country',
 'Ska',
 'Rap',
 'Indie / Alternative',
 'World',
 '¾Ë ¼ö ¾øÀ½',
 'World Folk',
 'Americana / Roots',
 'Alternatief',
 'New Wave',
 'Country | Blues | Folk',
 'Flemish Orchestral Melodic Pop',
 'Middle East / North Africa',
 'Roots / Americana',
 'Pop Punk',
 'Metal',

# Part 2: Merge track data and musical features per track

In [9]:
master_tracks.head()

,album_uri,num_tracks,track_number,explicit,track_name,track_type,track_duration_ms,track_disc_number,track_uri
0,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,1,False,Eraser,track,227426,1,spotify:track:7oolFzHipTMg2nL7shhdz2
1,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2,False,Castle on the Hill,track,261153,1,spotify:track:6PCUP3dWmTjcTtXY02oFdT
2,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,3,False,Dive,track,238440,1,spotify:track:51ChrwmUPDJvedPQnIU8Ls
3,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,4,False,Shape of You,track,233712,1,spotify:track:7qiZfU4dY1lWllzX7mPBI3
4,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,5,False,Perfect,track,263400,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v


In [10]:
master_track_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature
0,0.640,0.812,8,-5.647,0,0.0834,0.0860,0.000000,0.0509,0.914,86.013,audio_features,7oolFzHipTMg2nL7shhdz2,spotify:track:7oolFzHipTMg2nL7shhdz2,https://api.spotify.com/v1/tracks/7oolFzHipTMg...,https://api.spotify.com/v1/audio-analysis/7ool...,227427,4
1,0.461,0.834,2,-4.868,1,0.0989,0.0232,0.000011,0.1400,0.471,135.007,audio_features,6PCUP3dWmTjcTtXY02oFdT,spotify:track:6PCUP3dWmTjcTtXY02oFdT,https://api.spotify.com/v1/tracks/6PCUP3dWmTjc...,https://api.spotify.com/v1/audio-analysis/6PCU...,261154,4
2,0.761,0.386,4,-6.158,1,0.0399,0.3550,0.000000,0.0953,0.526,134.943,audio_features,51ChrwmUPDJvedPQnIU8Ls,spotify:track:51ChrwmUPDJvedPQnIU8Ls,https://api.spotify.com/v1/tracks/51ChrwmUPDJv...,https://api.spotify.com/v1/audio-analysis/51Ch...,238440,3
3,0.825,0.652,1,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
4,0.599,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050,audio_features,0tgVpDi06FyKpA1z0VMD4v,spotify:track:0tgVpDi06FyKpA1z0VMD4v,https://api.spotify.com/v1/tracks/0tgVpDi06FyK...,https://api.spotify.com/v1/audio-analysis/0tgV...,263400,3


In [13]:
print(master_tracks.track_uri.isin(master_track_features.track_uri).sum())
print(master_track_features.track_uri.isin(master_tracks.track_uri).sum())

6901
6901


In [15]:
track_level_data = pd.merge(master_tracks, master_track_features, how = 'inner', on = 'track_uri')
print(master_tracks.shape)
print(master_track_features.shape)
print(track_level_data.shape)

(6901, 9)
(6901, 18)
(6901, 26)


In [17]:
track_level_data.columns

Index(['album_uri', 'num_tracks', 'track_number', 'explicit', 'track_name',
       'track_type', 'track_duration_ms', 'track_disc_number', 'track_uri',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [16]:
track_level_data.head()

,album_uri,num_tracks,track_number,explicit,track_name,track_type,track_duration_ms,track_disc_number,track_uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,1,False,Eraser,track,227426,1,spotify:track:7oolFzHipTMg2nL7shhdz2,0.640,...,0.000000,0.0509,0.914,86.013,audio_features,7oolFzHipTMg2nL7shhdz2,https://api.spotify.com/v1/tracks/7oolFzHipTMg...,https://api.spotify.com/v1/audio-analysis/7ool...,227427,4
1,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2,False,Castle on the Hill,track,261153,1,spotify:track:6PCUP3dWmTjcTtXY02oFdT,0.461,...,0.000011,0.1400,0.471,135.007,audio_features,6PCUP3dWmTjcTtXY02oFdT,https://api.spotify.com/v1/tracks/6PCUP3dWmTjc...,https://api.spotify.com/v1/audio-analysis/6PCU...,261154,4
2,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,3,False,Dive,track,238440,1,spotify:track:51ChrwmUPDJvedPQnIU8Ls,0.761,...,0.000000,0.0953,0.526,134.943,audio_features,51ChrwmUPDJvedPQnIU8Ls,https://api.spotify.com/v1/tracks/51ChrwmUPDJv...,https://api.spotify.com/v1/audio-analysis/51Ch...,238440,3
3,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,4,False,Shape of You,track,233712,1,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.825,...,0.000000,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
4,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,5,False,Perfect,track,263400,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v,0.599,...,0.000000,0.1060,0.168,95.050,audio_features,0tgVpDi06FyKpA1z0VMD4v,https://api.spotify.com/v1/tracks/0tgVpDi06FyK...,https://api.spotify.com/v1/audio-analysis/0tgV...,263400,3


In [18]:
track_level_data.isnull().sum()

album_uri            0
num_tracks           0
track_number         0
explicit             0
track_name           0
track_type           0
track_duration_ms    0
track_disc_number    0
track_uri            0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
type                 0
id                   0
track_href           0
analysis_url         0
duration_ms          0
time_signature       0
dtype: int64

In [20]:
track_level_data.duplicated().sum()

0

In [21]:
track_level_data.track_uri.duplicated().sum()

0

# Part 3: Merge album level and track level data together

In [72]:
album_level_data_final.head()

,album_to_search,artist_to_search,artist_name_sp,artist_uri_sp,album_name_sp,album_uri,total_tracks_sp,release_date_sp,release_type_sp,album_name,popularity_sp,label_sp,genre
0,÷,ed sheeran,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop
1,remixed,nina simone,Nina Simone,spotify:artist:7G1GBhoKtEPnP86X2PvEYO,Remixed & Reimagined,spotify:album:0PJ70lJhWjflyKkrQabNso,16,2006-10-31,album,Remixed & Reimagined,41,RCA/Legacy,
2,routine maintenance,aaron west and the roaring twenties,Aaron West and The Roaring Twenties,spotify:artist:59cc2f0IvGu6YVEtY4cS0p,Routine Maintenance,spotify:album:5xAEfL4n6fBI5WZ9ugrmIw,10,2019-05-10,album,Routine Maintenance,41,Hopeless Records,
3,25,adele,Adele,spotify:artist:4dpARuHxo51G3z768sgnrY,25,spotify:album:0K4pIOOsfJ9lK8OjrZfXzd,11,2016-06-24,album,25,77,XL Recordings/Columbia,Pop
4,the impossible kid,aesop rock,Aesop Rock,spotify:artist:2fSaE6BXtQy0x7R7v9IOmZ,The Impossible Kid,spotify:album:1An1m0S3ZdQy9Uuo476D12,15,2016-04-29,album,The Impossible Kid,55,Rhymesayers,Rap & Hip-Hop


In [73]:
track_level_data.head()

,album_uri,num_tracks,track_number,explicit,track_name,track_type,track_duration_ms,track_disc_number,track_uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,1,False,Eraser,track,227426,1,spotify:track:7oolFzHipTMg2nL7shhdz2,0.640,...,0.000000,0.0509,0.914,86.013,audio_features,7oolFzHipTMg2nL7shhdz2,https://api.spotify.com/v1/tracks/7oolFzHipTMg...,https://api.spotify.com/v1/audio-analysis/7ool...,227427,4
1,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2,False,Castle on the Hill,track,261153,1,spotify:track:6PCUP3dWmTjcTtXY02oFdT,0.461,...,0.000011,0.1400,0.471,135.007,audio_features,6PCUP3dWmTjcTtXY02oFdT,https://api.spotify.com/v1/tracks/6PCUP3dWmTjc...,https://api.spotify.com/v1/audio-analysis/6PCU...,261154,4
2,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,3,False,Dive,track,238440,1,spotify:track:51ChrwmUPDJvedPQnIU8Ls,0.761,...,0.000000,0.0953,0.526,134.943,audio_features,51ChrwmUPDJvedPQnIU8Ls,https://api.spotify.com/v1/tracks/51ChrwmUPDJv...,https://api.spotify.com/v1/audio-analysis/51Ch...,238440,3
3,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,4,False,Shape of You,track,233712,1,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.825,...,0.000000,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
4,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,5,False,Perfect,track,263400,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v,0.599,...,0.000000,0.1060,0.168,95.050,audio_features,0tgVpDi06FyKpA1z0VMD4v,https://api.spotify.com/v1/tracks/0tgVpDi06FyK...,https://api.spotify.com/v1/audio-analysis/0tgV...,263400,3


In [66]:
print(album_level_data_final.shape)
print(track_level_data.shape)

(479, 13)
(6901, 26)


In [74]:
print(album_level_data_final.album_uri.isin(track_level_data.album_uri).sum())
print(track_level_data.album_uri.isin(album_level_data_final.album_uri).sum())

479
6901


In [75]:
omnibus_spotify_track_data = pd.merge(track_level_data, album_level_data_final, how = 'left', on = 'album_uri')
print(track_level_data.shape)
print(album_level_data_final.shape)
print(omnibus_spotify_track_data.shape)

(6901, 26)
(479, 13)
(6901, 38)


In [76]:
omnibus_spotify_track_data.isnull().sum()

album_uri              0
num_tracks             0
track_number           0
explicit               0
track_name             0
track_type             0
track_duration_ms      0
track_disc_number      0
track_uri              0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
type                   0
id                     0
track_href             0
analysis_url           0
duration_ms            0
time_signature         0
album_to_search        0
artist_to_search       0
artist_name_sp         0
artist_uri_sp          0
album_name_sp          0
total_tracks_sp        0
release_date_sp        0
release_type_sp        0
album_name             0
popularity_sp          0
label_sp               0
genre                531
dtype: int64

In [77]:
omnibus_spotify_track_data.head()

,album_uri,num_tracks,track_number,explicit,track_name,track_type,track_duration_ms,track_disc_number,track_uri,danceability,...,artist_name_sp,artist_uri_sp,album_name_sp,total_tracks_sp,release_date_sp,release_type_sp,album_name,popularity_sp,label_sp,genre
0,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,1,False,Eraser,track,227426,1,spotify:track:7oolFzHipTMg2nL7shhdz2,0.640,...,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop
1,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,2,False,Castle on the Hill,track,261153,1,spotify:track:6PCUP3dWmTjcTtXY02oFdT,0.461,...,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop
2,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,3,False,Dive,track,238440,1,spotify:track:51ChrwmUPDJvedPQnIU8Ls,0.761,...,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop
3,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,4,False,Shape of You,track,233712,1,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.825,...,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop
4,spotify:album:3T4tUhGYeRNVUGevb0wThu,16,5,False,Perfect,track,263400,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v,0.599,...,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,÷ (Deluxe),16,2017-03-03,album,÷ (Deluxe),89,Atlantic Records UK,Pop


In [ ]:
# save it out
out_dir = 'D:\\Data\\spotify_omnibus_data\\'
joblib.dump(omnibus_spotify_track_data, out_dir + 'omnibus_spotify_track_data.pkl')  

In [79]:
omnibus_spotify_track_data.to_csv(out_dir + 'omnibus_spotify_track_data.csv', index = False)